<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Model IO Exercise 

The purpose of this exercise is to test your understanding of building out Model IO systems. You will also hopefully notice the need to chain responses together, which we will cover later in this course!

Watch the video for a full overview on minimum outputs this class should be capable of, but feel free to expand on this project, or to just treat it as a code-along!

## History Quiz

Our main goal is to use LangChain and Python to create a very simple class with a few methods for:
* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer

### Suggested Imports

Feel free to accomplish this task however you prefer!

In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import datetime
from langchain.llms import OpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI

In [6]:
f = open('../openai_key.txt')
api_key = f.read()

In [63]:
class HistoryQuiz():
    
    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''
        # Guidance to llm
        system_template = "You are a historian who write quizzes on historical events involving {topic} that occurred."
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

        # What we want to achieve
        human_template = "Please give a quiz question requiring the date of occurrence as the answer. You only return the quiz question."

        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        # Input prompts to chat prompt template, then format the variable with inputs and apply to_messages to convert
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

        request = chat_prompt.format_prompt(topic=topic).to_messages()
        llm = ChatOpenAI(openai_api_key=api_key)

        result = llm(request)
        return result.content
    
    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''

        # Define parser required
        output_parser = DatetimeOutputParser()
        # Guidance to llm
        system_template = f"You are to provide the date as an answer to the question given"
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

        # What we want to achieve for response
        ai_template = "Please answer the given question: {question} \\n\\n in the format as follows: {format_instructions}"

        ai_message_prompt = AIMessagePromptTemplate.from_template(ai_template)

        # Input prompts to chat prompt template, then format the variable with inputs
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, ai_message_prompt])

        # Format the chat prompt with input variables:  question and format_instructions (datetimeoutput parser) 
        request = chat_prompt.format_prompt(
            question=question,
            format_instructions=output_parser.get_format_instructions()
        ).to_messages()

        # Use ChatOpenAI to provide completion based on formatted chat prompt
        llm = ChatOpenAI(openai_api_key=api_key)
        correct_datetime = llm(request).content
        
        # Attempt to parse second round incase prompt template fails
        correct_datetime = output_parser.parse(correct_datetime)
        return correct_datetime
    
    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        year = input("Enter the year:")
        mth = input("Enter the month (1-12):")
        day = input("Enter the day (1-31):")

        dt = datetime(year=int(year), month=int(mth), day=int(day))
        return dt
        
        
    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        # print or return the difference between the answers here!
        
        # Stringify difference in dates
        date_diff = str(user_answer - ai_answer)

        return_str =  f"The difference between the dates is: {date_diff}"
        
        return return_str

### Example Usage

Feel free to expand or edit this project. To keep things simple we have every method return an object that will then feed into a new method!

In [64]:
quiz_bot = HistoryQuiz()

In [65]:
question = quiz_bot.create_history_question(topic='World War 2')

In [66]:
question

'On what date did the Allied forces launch the D-Day invasion of Normandy during World War II?'

In [67]:
ai_answer = quiz_bot.get_AI_answer(question)

In [68]:
# Day After Pearl Harbor
ai_answer

datetime.datetime(1944, 6, 6, 0, 0)

In [69]:
user_answer = quiz_bot.get_user_answer(question)

In [70]:
user_answer

datetime.datetime(1944, 6, 1, 0, 0)

In [71]:
quiz_bot.check_user_answer(user_answer,ai_answer)

'The difference between the dates is: -5 days, 0:00:00'